In [1]:
import re
import string
import keras
import tensorflow as tf
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Flatten, Input, Activation, Dropout
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from keras.layers import AveragePooling2D, BatchNormalization

Using TensorFlow backend.


In [2]:
sess = tf.Session(config=tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2}))
keras.backend.set_session(sess)

In [3]:
df = pd.read_csv('./IMDB Dataset.csv')
df.review = df.review.apply(lambda x: BeautifulSoup(x).get_text())
df.review = df.review.apply(lambda x: x.lower())
df.review = df.review.apply(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
df.review = df.review.apply(lambda x: x.translate(x.maketrans('', '', string.digits)))
df.sentiment = df.sentiment.apply(lambda x : 0 if (x == 'negative') else 1.0)
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,1.0
1,a wonderful little production the filming tech...,1.0
2,i thought this was a wonderful way to spend ti...,1.0
3,basically theres a family where a little boy j...,0.0
4,petter matteis love in the time of money is a ...,1.0
5,probably my alltime favorite movie a story of ...,1.0
6,i sure would like to see a resurrection of a u...,1.0
7,this show was an amazing fresh innovative ide...,0.0
8,encouraged by the positive comments about this...,0.0
9,if you like original gut wrenching laughter yo...,1.0


In [4]:
max_features = 20000
maxlen = 200  # cut texts after this number of words (among top max_features most common words)
batch_size = 256
embedding_dim = 256

In [5]:
X_train, X_test, y_train,y_test = train_test_split(df.review, df.sentiment, test_size=0.2)

In [6]:
tokenizer = Tokenizer(num_words=6000)
tokenizer.fit_on_texts(X_train) # Fit the tokenizer on the text

X_train = tokenizer.texts_to_sequences(X_train) # Convert the text to sequences
X_test = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [7]:
model = Sequential()

In [8]:
model.add(Embedding(max_features, 512,input_shape=(maxlen,),trainable=True))
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 512)          10240000  
_________________________________________________________________
flatten_1 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               13107328  
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [10]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=10)# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train,batch_size=batch_size,epochs=20,validation_split=0.1, callbacks=[es])
score, acc = model.evaluate(X_test, y_test ,batch_size=batch_size)
score, acc

Instructions for updating:
Use tf.cast instead.
Train on 36000 samples, validate on 4000 samples
Epoch 1/20
36000/36000 [==============================] - 19s 526us/step - loss: 0.6935 - acc: 0.6439 - val_loss: 0.3551 - val_acc: 0.8415
Epoch 2/20
36000/36000 [==============================] - 13s 368us/step - loss: 0.2209 - acc: 0.9151 - val_loss: 0.3733 - val_acc: 0.8562
Epoch 3/20
36000/36000 [==============================] - 13s 374us/step - loss: 0.0309 - acc: 0.9922 - val_loss: 0.6609 - val_acc: 0.8327
Epoch 4/20
36000/36000 [==============================] - 14s 382us/step - loss: 0.0050 - acc: 0.9993 - val_loss: 0.6066 - val_acc: 0.8485
Epoch 5/20
36000/36000 [==============================] - 14s 388us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.6250 - val_acc: 0.8508
Epoch 6/20
36000/36000 [==============================] - 14s 393us/step - loss: 5.9936e-04 - acc: 1.0000 - val_loss: 0.6370 - val_acc: 0.8480
Epoch 7/20
36000/36000 [==============================] - 14s 391

(0.6160360527813434, 0.8579)

In [12]:
import numpy as np
import keras.preprocessing.text

text = np.array(['this is just some random, stupid text, This is bloody random anyway so I am really trying to predict'])
tokenizer.fit_on_texts(text)
pred = tokenizer.texts_to_sequences(text)
pred = np.array(pred)
pred

# predictions = model.predict(pred)

array([[  10,    6,   39,   45, 1399,  358, 2979,   10,    6, 1650, 1399,
         660,   37,    9,  225,   60,  253,    5, 5824]])